# Imports and Setup

In [2]:
import os
from pprint import pprint
import json
import random
import numpy as np
from helpers.helper import get_cath

from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio.Seq import Seq
from Bio import SeqIO

import requests
from requests.packages.urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import shutil
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio.Seq import Seq
from Bio import SeqIO


from scipy.stats import ttest_ind
from pprint import pprint

cath = get_cath()


In [3]:
# read the reverse mapping
with open('../data/sword2/SWORD2/misc/new_iid/pdb_to_uniprot_map.json') as json_file:
    pdb_to_uni_map = json.load(json_file)

In [4]:
pdb_to_af_map = {}
for elt in pdb_to_uni_map['results']:
    pdb = elt['from']
    databases = [db['database'] for db in elt['to']['uniProtKBCrossReferences']]
    for i, db in enumerate(databases):
        if db == 'AlphaFoldDB':
            af_id = (elt['to']['uniProtKBCrossReferences'][i]['id'])
            if pdb_to_af_map.get(pdb):
                pdb_to_af_map[pdb].append(af_id)
            else:
                pdb_to_af_map[pdb] = [af_id]
    

In [5]:
total = 0
for k, v in pdb_to_af_map.items():
	total += len(v)
print("Total:", total)

Total: 10005


In [32]:
# download AF PDB files
counter = 0
for _, codes in pdb_to_af_map.items():
	for code in codes:
		(bool, msg) = download_af_model(code)
		if not bool:
			print(msg)
		counter += 1
		if counter % 250 == 0:
			print(f"[{counter}/{total}]")
print(f"[{counter}/{total}]")

[250/10005]
[500/10005]
[750/10005]
[1000/10005]
[1250/10005]
[1500/10005]
[1750/10005]
[2000/10005]
[2250/10005]
[2500/10005]
[2750/10005]
[3000/10005]
[3250/10005]
[3500/10005]
[3750/10005]
[4000/10005]
[4250/10005]
[4500/10005]
[4750/10005]
[5000/10005]
[5250/10005]
[5500/10005]
[5750/10005]
[6000/10005]
[6250/10005]
[6500/10005]
[6750/10005]
[7000/10005]
[7250/10005]
[7500/10005]
[7750/10005]
[8000/10005]
[8250/10005]
[8500/10005]
[8750/10005]
[9000/10005]
[9250/10005]
[9500/10005]
[9750/10005]
[10000/10005]
[10005/10005]


In [11]:
with open('../data/cath/iid/chains_to_seq_iid.json') as json_file:
    chain_to_seq_iid = json.load(json_file)

valid_pairs = []
for chain, seq in chain_to_seq_iid.items():
    if pdb_to_af_map.get(chain[:4]):
        pdb_filename = f"../data/pdb/new_iid/{chain[:4]}.pdb"
        chain_id = chain[-1]
        with open(pdb_filename, "r") as pdb_file:
            pdb_chains = {record.id: record.seq for record in SeqIO.parse(pdb_file, 'pdb-seqres')}
            for key in pdb_chains.keys():
                if key[-1] == chain[-1]:
                    pdb_seq = pdb_chains[key]
        af_seqs = []
        for uniprot in pdb_to_af_map.get(chain[:4]):
            af_seq = get_af_chain(uniprot, chain[-1])
            if af_seq:
                af_seqs.append((uniprot, af_seq))
        for (uniprot, af_seq) in af_seqs:
            if len(af_seq) == len(pdb_seq):
                sim = sequence_sim(af_seq, pdb_seq)
                if sim == 1.0:
                    valid_pairs.append((chain, uniprot + f':{chain[-1]}'))



In [12]:
for (a, b) in valid_pairs:
    if b[-1] != 'A':
        print(a,b)
        
# These ARE ALL A chains

In [83]:
valid_pdbs = [x[0] for x in valid_pairs]
valid_afs = [x[1] for x in valid_pairs]

from collections import Counter
for k,v in Counter(valid_pdbs).items():
    if v > 1:
        print(k, v)

1p4x:A 2
1s7o:A 2


In [88]:
# write the pairs to their respective files
with open('../data/sword2/SWORD2/misc/new_iid/sword_af.txt', "w") as af_f:
    with open('../data/sword2/SWORD2/misc/new_iid/sword_pdb.txt', "w") as pdb_f:
        for af in set(valid_afs):
            af_f.write(af[:-2] + '\n')
        for pdb in set(valid_pdbs):
            pdb_f.write(pdb[:-2] + '\n')


In [ ]:
# write a true_random_sample_keys.txt
with open('../data/cath/iid/true_random_sample_keys.txt', 'w') as f:
    for chain in chain_to_seq_iid.keys():
        f.write(chain + '\n')

In [93]:
# analyse time estimate
minutes = []
with open("../data/sword2/SWORD2/misc/new_iid/estimate_time.txt") as f:
	lines = f.readlines()
	for i in range(0, len(lines), 3):
		_, mins, id = lines[i], lines[i+1], lines[i+2]
		mins = mins.split('    ')
		mins = int(mins[1].strip())
		id = id.split('    ')
		id = id[1].strip()
		minutes.append((mins, id))

In [98]:
just_minutes = [x[0] for x in minutes]
np.mean(just_minutes)
len(just_minutes)

454

In [57]:
valid_pairs[:40]

[('1shs', 'P19632'),
 ('1shs', 'P61972'),
 ('1shs', 'P68699'),
 ('1shs', 'P45600'),
 ('1shs', 'P37064'),
 ('1shs', 'Q05506'),
 ('1shs', 'P33447'),
 ('1shs', 'P00968'),
 ('1shs', 'Q9UVT8'),
 ('1shs', 'P59936'),
 ('1shs', 'P38636'),
 ('1shs', 'P10618'),
 ('1shs', 'Q08602'),
 ('1shs', 'Q08603'),
 ('1shs', 'Q9REC4'),
 ('1shs', 'Q56308'),
 ('1shs', 'P28298'),
 ('1shs', 'Q05581'),
 ('1shs', 'P95474'),
 ('1shs', 'P31101'),
 ('1shs', 'Q9ZBA9'),
 ('1shs', 'O76463'),
 ('1shs', 'P80176'),
 ('1shs', 'Q56694'),
 ('1shs', 'Q50744'),
 ('1shs', 'P06149'),
 ('1shs', 'P0ABJ1'),
 ('1shs', 'P29208'),
 ('1shs', 'Q8ZYK1'),
 ('1shs', 'P72322'),
 ('1shs', 'P44853'),
 ('1shs', 'P20356'),
 ('1shs', 'P96142'),
 ('1shs', 'P03012'),
 ('1shs', 'Q9WY48'),
 ('1shs', 'Q57840'),
 ('1shs', 'P81156'),
 ('1shs', 'P62993'),
 ('1shs', 'P04425'),
 ('1shs', 'P9WQB5')]

# Functions

## SWORD parsing

In [6]:
def get_sword2(code, version, verb=False):
    file = f"../data/sword2/SWORD2/results/{version}/{code}/{code}_A/sword.txt"
    with open(file, "r") as f:
        data = {}
        lines = f.readlines()
        option = 0
        for i, line in enumerate(lines):
            lines[i] = "".join([c for c in line if c not in ["\n",'']])
            if line != "\n":
                if not line.startswith(("PDB:", "#D", "A")):
                    res = lines[i].split("|")
                    boundaries = res[2]
                    domains = boundaries.strip().split(" ")
                    data[f"option{option}"] = {}
                    for j in range(len(domains)):
                        data[f"option{option}"][str(j+1)] = domains[j]
                    option += 1
    verb and pprint(data)
    return data

## Metrics

In [7]:
def sequence_sim(seq1, seq2, match_score = 1, mismatch_score = -1, gap_penalty = -2):
	alignments = pairwise2.align.globalxx(seq1, seq2)

	# Print the alignment(s)
	# for alignment in alignments:
		# print(format_alignment(*alignment))
	score = alignments[0].score
	norm_score = score / max(len(seq1), len(seq2))
	return norm_score



def dbd_score(y_pred, y_true, margin=20):
    scores = []
    for i in range(len(y_pred)):
        window = y_true[max(0, i-margin):min(len(y_true), i+margin+1)]
        indices_window = list(range(max(0, i-margin), min(len(y_true), i+margin+1)))
        if y_pred[i] == 1.0:
            if 1.0 in window:
                # if it's within the window, calculate the score
                pos = np.where(window == 1.0)[0][0]
                j = indices_window[pos]
                diff = abs(i - j)
                k = 0 if diff == 0 else 1
                score = ((margin - diff) + k) / margin
            else:
                # false positive
                score = 0
            scores.append(score)

    number_of_true_boundaries = np.sum(y_true)
    number_of_pred_boundaries = np.sum(y_pred)
    max_len = max(number_of_true_boundaries,number_of_pred_boundaries)
    if max_len == 0:
        return 1.0

    return np.sum(scores) / max_len


def observations(y_pred, y_true, margin):
    tp = 0
    tn = 0
    fp = 0
    fn = 0

    dbd = dbd_score(y_pred, y_true, margin)
    for i in range(len(y_pred)):
        window = y_true[max(0, i-margin):min(len(y_true), i+margin+1)]
        indices_window = list(range(max(0, i-margin), min(len(y_true), i+margin+1)))
        if y_pred[i] == 1.0:
            if 1.0 in window:
                pos = np.where(window == 1.0)[0][0]
                j = indices_window[pos]
                y_true[j] = 0.0
                tp += 1
            else:
                fp += 1


        elif y_pred[i] == 0.0:
            if  y_true[i] == 1.0:
                fn += 1
            else:
                tn += 1

    return (tp, tn, fp, fn)


def observations__(y_pred, y_true, margin):
    tp = 0
    tn = 0
    fp = 0
    fn = 0

    dbd = dbd_score(y_pred, y_true, margin)
    for i in range(len(y_pred)):
        window = y_true[max(0, i-margin):min(len(y_true), i+margin+1)]
        indices_window = list(range(max(0, i-margin), min(len(y_true), i+margin+1)))
        if y_pred[i] == 1.0:
            if 1.0 in window:
                pos = np.where(window == 1.0)[0][0]
                j = indices_window[pos]
                y_true[j] = 0.0
                tp += 1
            else:
                fp += 1

    for i in range(len(y_pred)):
        if y_pred[i] == 0.0:
            if  y_true[i] == 1.0:
                fn += 1
            else:
                tn += 1

    return (tp, tn, fp, fn)


def metrics(y_pred, y_true, margin=20):
    tp, tn, fp, fn = observations__(y_pred, y_true, margin)

    accuracy = (tn + tp) / (tn + tp + fn + fp) if (tn + tp + fn + fp) else 0
    precision = tp / (tp + fp) if (tp + fp) else 0
    recall = tp / (tp + fn) if (tp + fn) else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) else 0

    mcc_num = (tp * tn) - (fp * fn)
    mcc_den = np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    mcc = mcc_num / mcc_den if mcc_den else 0

    dbd = dbd_score(y_pred, y_true, margin)

    return (accuracy, precision, recall, f1, mcc, dbd)

## Other

In [8]:
def requests_retry_session(retries=3,
                           backoff_factor=0.3,
                           status_forcelist=(500, 502, 504),
                           session=None):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

def download_af_model(id):
    """
    Download the Alphafold2 model corresponding to the Uniprot Id given by user
    https://alphafold.ebi.ac.uk/

    Returns:
        - File path (string): Path of the downloaded PDB file
        or
        False if wrong id
        "DOWNLOAD ERROR" if could not download
    """
    RESULTS_DIR = "../data/sword2/SWORD2/misc/new_iid/af_pdbs/"
    name = f"AF-{id}-F1-model_v3"
    url = f"https://alphafold.ebi.ac.uk/files/{name}.pdb"
    try:
        response = requests_retry_session().get(url)
    except Exception as x:
        return (False, x)
    with open(f"{RESULTS_DIR}/{name}.pdb", "w") as f:
        f.write(response.text)
    return (True, f"{RESULTS_DIR}/{name}.pdb")

## Boundaries

In [9]:
def boundaries2(len_seq, domain, discontinuity_delimiter):
	"""
		Defines a boundary as the beginning of a domain ONLY in multi-domain proteins
	"""
	first_start = np.inf
	bounds = np.zeros((len_seq), dtype=np.int8)
	for k, v in domain.items():
		boundary_positions = v.split(discontinuity_delimiter)
		for b in boundary_positions:
			start, end = [int(i) for i in b.split('-')]
			if start < first_start:
				first_start = start
			bounds[start-1] = 1
	bounds[first_start-1] = 0            
	return np.array(bounds, dtype=np.bool_)

In [10]:
def get_af_chain(code, chain):
	file_path = f"../data/sword2/SWORD2/misc/new_iid/af_pdbs/AF-{code}-F1-model_v3.pdb"
	chains = {record.id: record.seq for record in SeqIO.parse(file_path, 'pdb-seqres')}
	seq = chains.get(f'XXXX:{chain}')
	return seq


def get_pdb_chain(code, chain):
	pdb_file_path = f"../data/pdb/new_iid/{code}.pdb"
	pdb_chains = {record.id: record.seq for record in SeqIO.parse(pdb_file_path, 'pdb-seqres')}

	for key in pdb_chains.keys():
		if key[-1] == 'A':
			a_chain_pdb_seq = pdb_chains[key]
			return a_chain_pdb_seq

In [7]:
ypr = np.array([0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0], dtype=np.float16)
ytr = np.array([0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0], dtype=np.float16)

mcc = metrics(ypr, ytr)[-2]
mcc

0.6882472016116853

# Analysis

In [100]:
path = '../data/sword2/final_results/results/af'

uniprots = os.listdir(path)
# print(uniprots)
# print()

pdb_mcc = []
af_mcc = []

pdb_f1 = []
af_f1 = []

pdb_no_bounds = []
af_no_bounds = []

for i, id in enumerate(uniprots):

    pdb = pdb_uniprot_mappings_reverse[id]
    a_chain_uniprot_seq = get_af_chain(id)
    a_chain_pdb_seq = get_pdb_chain(pdb)
    chain_len = None
    if len(a_chain_pdb_seq) != len(a_chain_uniprot_seq):
        raise ValueError("Different sequence lengths is not expected")
    else:
        chain_len = len(a_chain_pdb_seq)
    
    baseline = cath[pdb]['A']
    af_sword_results = get_sword2(id, 'af', verb=False)
    try:
        pdb_sword_results = get_sword2(pdb, 'pdb', verb=False)
    except FileNotFoundError:
        print("File not found", id, pdb)
        continue

    margin = 20
    baseline_boundaries = boundaries2(len(a_chain_pdb_seq), baseline, ',').astype(int)
    pdb_mccs = []
    af_mccs = []
    pdb_dbds = []
    af_dbds = []
    pdb_f1s = []
    af_f1s = []
    pdb_bounds = []
    af_bounds = []

    for option, domain in pdb_sword_results.items():
        pdb_sword_boundaries = boundaries2(chain_len, domain, ';').astype(int)
        pdb_sword_metrics = metrics(pdb_sword_boundaries, baseline_boundaries, margin)
        pdb_sword_mcc = pdb_sword_metrics[-2]
        pdb_sword_dbd = pdb_sword_metrics[-1]
        pdb_sword_f1 = pdb_sword_metrics[-3]
        pdb_mccs.append(pdb_sword_mcc)
        pdb_dbds.append(pdb_sword_dbd)
        pdb_f1s.append(pdb_sword_f1)
        pdb_bounds.append(np.sum(pdb_sword_boundaries))

    for option, domain in af_sword_results.items():
        af_sword_boundaries = boundaries2(chain_len, domain, ';').astype(int)
        af_sword_metrics = metrics(af_sword_boundaries, baseline_boundaries, margin)
        af_sword_mcc = af_sword_metrics[-2]
        af_sword_dbd = af_sword_metrics[-1]
        af_sword_f1 = af_sword_metrics[-3]
        af_mccs.append(af_sword_mcc)
        af_dbds.append(af_sword_dbd)
        af_f1s.append(af_sword_f1)
        af_bounds.append(np.sum(af_sword_boundaries))

    af_f1.append(max(af_f1s))
    pdb_f1.append(max(pdb_f1s))

    best_pdb_i = np.argmax(pdb_mccs)
    best_af_i = np.argmax(af_mccs)
    pdb_no_bounds.append(pdb_bounds[best_pdb_i])

    pdb_mcc.append(pdb_mccs[best_pdb_i])
    af_mcc.append(af_mccs[best_af_i])
    af_no_bounds.append(af_bounds[best_af_i])

    if (i + 1) % 50 == 0:
        print(f"[{i + 1}/{len(uniprots)}]")

print(f"[{i + 1}/{len(uniprots)}]")


[50/443]
[100/443]
[150/443]
[200/443]
[250/443]
File not found Q9YDZ4 2hls
[300/443]
[350/443]
File not found P38505 1jfj
[400/443]
[443/443]


In [101]:
print("af", np.mean(af_no_bounds))
print("pdb", np.mean(pdb_no_bounds))

af 2.2448979591836733
pdb 2.941043083900227


In [97]:
t_stat, p_val = ttest_ind(pdb_mcc, af_mcc)

print("Mean PDB MCC:", np.mean(pdb_mcc))
print("Mean AF MCC:", np.mean(af_mcc))

print(t_stat, float(p_val))

Mean PDB MCC: 0.6210332441611623
Mean AF MCC: 0.13217732495763904
27.3760386666944 7.451803918104624e-120


In [102]:
t_stat, p_val = ttest_ind(pdb_mcc, af_mcc)

print("Mean PDB MCC:", np.mean(pdb_mcc))
print("Mean AF MCC:", np.mean(af_mcc))

print(t_stat, float(p_val))

Mean PDB MCC: 0.6740580770787745
Mean AF MCC: 0.1436396457669393
28.077329120492674 2.2876669819673796e-124


In [98]:
t_stat, p_val = ttest_ind(af_no_bounds, pdb_no_bounds)

print("Mean PDB MCC:", np.mean(pdb_no_bounds))
print("Mean AF MCC:", np.mean(af_no_bounds))

print(t_stat, float(p_val))

Mean PDB MCC: 2.9115646258503403
Mean AF MCC: 2.240362811791383
-5.973230598758096 3.373535054941592e-09


Mean PDB MCC: 0.7312201785001735

Mean AF MCC: 0.04587217486107879

56.20185031552613 2.0480532531702836e-293

In [ ]:
# import tensorflow as tf
# from keras import backend as K

# def sensitivity(y_true, y_pred):
#     true_label = K.argmax(y_true, axis=-1)
#     pred_label = K.argmax(y_pred, axis=-1)
#     INTERESTING_CLASS_ID = 2
#     sample_mask = K.cast(K.not_equal(true_label, INTERESTING_CLASS_ID), 'int32')

#     TP_tmp1 = K.cast(K.equal(true_label, 0), 'int32') * sample_mask
#     TP_tmp2 = K.cast(K.equal(pred_label, 0), 'int32') * sample_mask    
#     TP = K.sum(TP_tmp1 * TP_tmp2)

#     FN_tmp1 = K.cast(K.equal(true_label, 0), 'int32') * sample_mask
#     FN_tmp2 = K.cast(K.not_equal(pred_label, 0), 'int32') * sample_mask    
#     FN = K.sum(FN_tmp1 * FN_tmp2)

#     epsilon = 0.000000001
#     return K.cast(TP, 'float') / (K.cast(TP, 'float') + K.cast(FN, 'float') + epsilon)


# def precision(y_true, y_pred):
#     true_label = K.argmax(y_true, axis=-1)
#     pred_label = K.argmax(y_pred, axis=-1)
#     INTERESTING_CLASS_ID = 2
#     sample_mask = K.cast(K.not_equal(true_label, INTERESTING_CLASS_ID), 'int32')

#     TP_tmp1 = K.cast(K.equal(true_label, 0), 'int32') * sample_mask
#     TP_tmp2 = K.cast(K.equal(pred_label, 0), 'int32') * sample_mask
#     TP = K.sum(TP_tmp1 * TP_tmp2)

#     FP_tmp1 = K.cast(K.not_equal(true_label, 0), 'int32') * sample_mask
#     FP_tmp2 = K.cast(K.equal(pred_label, 0), 'int32') * sample_mask
#     FP = K.sum(FP_tmp1 * FP_tmp2)

#     epsilon = 0.000000001
#     return K.cast(TP, 'float') / (K.cast(TP, 'float') + K.cast(FP, 'float') + epsilon)

# def f1_score(y_true, y_pred):
#     pre = precision(y_true, y_pred)
#     sen = sensitivity(y_true, y_pred)
#     epsilon = 0.000000001
#     f1 = 2 * pre * sen / (pre + sen + epsilon)
#     return f1